In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
import os
import cv2

args = {
    "image" : "../input/multi-camera-person-tracking-reduced-frames/reduced_frames/campus/reduced_frames3/40.jpg",
    "yolo" : "../input/yolo-coco-data",
    "confidence" : 0,
    "threshold" : 0
}

labelsPath = os.path.sep.join([args["yolo"], "coco.names"])
LABELS = open(labelsPath).read().strip().split("\n")
# initialize a list of colors to represent each possible class label
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
	dtype="uint8")
# derive the paths to the YOLO weights and model configuration
weightsPath = os.path.sep.join([args["yolo"], "yolov3.weights"])
configPath = os.path.sep.join([args["yolo"], "yolov3.cfg"])
# load our YOLO object detector trained on COCO dataset (80 classes)
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

bboxes_campus = {}

def yolo(path):
    # load our input image and grab its spatial dimensions
    path = path
    image = cv2.imread(path)
    (H, W) = image.shape[:2]
    # determine only the *output* layer names that we need from YOLO
    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    # construct a blob from the input image and then perform a forward
    # pass of the YOLO object detector, giving us our bounding boxes and
    # associated probabilities
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),
        swapRB=True, crop=False)
    net.setInput(blob)
    start = time.time()
    layerOutputs = net.forward(ln)
    end = time.time()
    # show timing information on YOLO
#     print("[INFO] YOLO took {:.6f} seconds".format(end - start))
    
    # initialize our lists of detected bounding boxes, confidences, and
    # class IDs, respectively
    boxes = []
    confidences = []
    classIDs = []
    
    # loop over each of the layer outputs
    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            # extract the class ID and confidence (i.e., probability) of
            # the current object detection
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            # filter out weak predictions by ensuring the detected
            # probability is greater than the minimum probability
            if confidence > args["confidence"]:
                # scale the bounding box coordinates back relative to the
                # size of the image, keeping in mind that YOLO actually
                # returns the center (x, y)-coordinates of the bounding
                # box followed by the boxes' width and height
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                # use the center (x, y)-coordinates to derive the top and
                # and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                # update our list of bounding box coordinates, confidences,
                # and class IDs
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)
    # apply non-maxima suppression to suppress weak, overlapping bounding
    # boxes
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, args["confidence"],
        args["threshold"])
    # ensure at least one detection exists
    bbox = []
    if len(idxs) > 0:
        bbox = []
        for i in idxs.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            # draw a bounding box rectangle and label on the image
            color = [int(c) for c in COLORS[classIDs[i]]]
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
            cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            bbox.append([classIDs[i], x, y, w, h ])
    # show the output image
    cv2.imwrite("Image1.png", image)
    bboxes_campus[path.split('/')[-1].split('.')[0]] = bbox
#     print(bboxes_campus)
    return bboxes_campus 

[INFO] loading YOLO from disk...


KeyboardInterrupt: 

In [ ]:
import time
yolo(args['image'])

In [ ]:
import time

## Getting the reduced frames for each of the diven directories

directories = ['campus','lab']
output_dir = './reduced_frames3'

dir_files = {}

for directory in directories:
    parent_path = '../input/multi-camera-person-tracking-reduced-frames/reduced_frames/' + directory + "/"
    dirs = os.listdir(parent_path)
    needed = 'asdasd'
    for item in dirs:
        print(os.path.join(parent_path,item))
        if(os.path.isdir(os.path.join(parent_path,item))):
            needed = item
#     print(needed)
    
    for dirname, _, filenames in os.walk('../input/multi-camera-person-tracking-reduced-frames/reduced_frames/' + directory + "/" + needed + "/"):
        files = []
        filenames = map(lambda x: x.split(".")[0],filenames)
        for filename in sorted(filenames,key=int):
            files.append(os.path.join(dirname, filename + ".jpg"))
            print(os.path.join(dirname, filename + ".jpg"))
        dir_files[directory] = files
        
# out = yolo(args['image'])

In [ ]:
#Finding BBoxes of different coco ClassID's in our image and storing their BBox 
bboxes_all = {}
for key in dir_files:
    bboxes_campus = {}
    print('STARTED ' + key)
    for file in dir_files[key]:
        out = yolo(file)
        print(len(out),"-",list(out.keys())[-1],end = "-")
        print()
    bboxes_all[key] = out
    print(bboxes_all)

In [ ]:
bboxes_all


In [ ]:
# Dumping this dict of scene and BBoxes into a pickle file

import pickle

with open('filename.pickle', 'wb') as handle:
    pickle.dump(bboxes_all, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
from IPython.display import FileLink

FileLink(r'filename.pickle')